In [1]:
import numpy as np

# 第２ステージ　自然なコードで表現する　

## ステップ１１　可変長の引数（順伝播編）

### 11.1 Functionクラスの修正
現状のFunctionクラスを、複数の入出力に対応できるよう修正する。<br>
以下、現状のFunctionクラス
```Python
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()
```

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [3]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  # Get data from Variable
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


### 11.2 Addクラスの実装

In [5]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする

In [6]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs) # ysはタプル
y = ys[0]
print(y.data)

5


## ステップ１２　可変長の引数（改善編）

### 12.1 １つ目の改善：関数を使いやすく
まずは、関数を<strong>使う人</strong>にとっての改善を行う。<br>
- 現状のコード<br>
```Python
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
```
- 改善後のコード<br>
```Python
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)
```

In [7]:
class Function:
    def __call__(self, *inputs): # ①アスタリスクを付ける
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        # ②リストの要素が１つの時は最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

- memo<br>
<strong>可変長引数</strong><br>
変数にアスタリスクを付けることで、リストを使わずに任意の数の引数を与えて、<br>
関数を呼ぶことができる。以下に具体例を挙げる。

In [8]:
def print_x(*x):
    print(x)
    
print_x(1, 2, 3)

a, b = (1, 2), (3, 4)
print_x(a, b)

(1, 2, 3)
((1, 2), (3, 4))


In [9]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする

In [10]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


### 12.2 ２つ目の改善：関数を実装しやすく
今度は、関数を<strong>実装する人</strong>にとっての改善を行う。<br>
具体的には、メソッドの引数は直接受け取り、結果の変数も直接返す。<br>
- 現状のコード
```Python
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする
```
- 改善後のコード
```Python
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
```

In [11]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # ①アスタリスクを付けてアンパッキング
        if not isinstance(ys, tuple): # ②タプルではない場合の追加対応
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 12.3 add関数の実装
Addクラスを<strong>Pythonの関数</strong>として利用できるようにする。

In [12]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

In [13]:
def add(x0, x1):
    return Add()(x0, x1)

In [14]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## ステップ１３　可変長の引数（逆伝播編）

### 13.1 可変長引数に対応したAddクラスの逆伝播
ステップ１２の通り、足し算の順伝播は入力が２つ、出力が１つである。<br>
逆伝播時は、逆に、入力が１つ、出力が２つになる。<br>
例えば、$y=x_0+x_1$の場合、
$$\frac{\partial y}{\partial x_0} = 1, \frac{\partial y}{\partial x_1} = 1$$
であることから、足し算の逆伝播は、上流の微分を<strong>そのまま流す</strong>といいことがわかる。

In [15]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

### 13.2 Variableクラスの修正
- 現状のVariableクラス
```Py
class Variable:
    ...
    def backward(self):
    if self.grad is None:
        self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
        f = funcs.pop()
        x, y = f.input, f.output # ①関数の入出力を取得 
        x.grad = f.backward(y.grad) # ②backwardメソッドを呼ぶ

        if x.creator is not None:
            funcs.append(x.creator)
```
現状では、①において、関数の入出力を１つだけに限定している。<br>
これを複数の変数に対応できるように4つの修正を加える。

In [16]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] # ①
            gxs = f.backward(*gys) # ②
            if not isinstance(gxs, tuple): # ③
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # ④
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

①の箇所で、出力変数であるoutputsの微分をリストにまとめる。<br>
そして②において、関数fの逆伝播を呼び出す。ここで、変数に*をつけて、<br>
リストのアンパック（展開）を行う。<br>
また、③の箇所では、gxsがタプルではない時に、それをタプルへと変換する。
<br>
④では、各入力とそれに対応する微分のペアを設定する。


### 13.3 Squareクラスの実装

In [17]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 修正前はx = self.input.data
        gx = 2 * x * gy
        return gx

In [18]:
def square(x):
    f = Square()
    return f(x)

In [19]:
def add(x0, x1):
    return Add()(x0, x1)

In [20]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


- memo<br>
上記のプログラムでは、$z=x^2+y^2$という計算を行った。<br>
`z.backward()`の部分の計算過程を図にしたものが以下のものである。<br>
<img src="image/step13_ex.png" width="800" />